In [ ]:
!pip install yt-dlp
!pip install moviepy
!pip install torch torchvision torchaudio
!pip install git+https://github.com/openai/whisper.git



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.6/171.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.5 MB/s eta 0:00:00


In [ ]:
from yt_dlp import YoutubeDL
from moviepy.editor import VideoFileClip
import os
from glob import glob
import whisper
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from transformers import BertTokenizer, EncoderDecoderModel
import re

In [ ]:
video_url = ['https://youtu.be/fC7V8QsPBec?si=4ooByD-C_Kdlrwh2','https://youtu.be/a2B69vCtjOU?si=e4V3gGPYSnuNT7p7',"https://youtu.be/otN3V0beVRo?si=-Zq2eyfFalDmES31","https://youtu.be/Gm4W6BNqDB4?si=06oKDIFz9zbjkfZG"]


In [ ]:
def download_youtube_video_yt_dlp(url, output_path='/content/videos/'):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': f'{output_path}/%(title)s.%(ext)s'
    }

    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

In [ ]:
for i in video_url:
  download_youtube_video_yt_dlp(i)

[youtube] Extracting URL: https://youtu.be/fC7V8QsPBec?si=4ooByD-C_Kdlrwh2
[youtube] fC7V8QsPBec: Downloading webpage
[youtube] fC7V8QsPBec: Downloading ios player API JSON
[youtube] fC7V8QsPBec: Downloading mweb player API JSON
[youtube] fC7V8QsPBec: Downloading player 4e23410d
[youtube] fC7V8QsPBec: Downloading m3u8 information
[info] fC7V8QsPBec: Downloading 1 format(s): 609+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 181
[download] Destination: /content/videos//Week 1 - Lecture 1 - Introduction to Machine Learning.f609.mp4
[download] 100% of  110.34MiB in 00:00:13 at 7.89MiB/s                   
[download] Destination: /content/videos//Week 1 - Lecture 1 - Introduction to Machine Learning.f140.m4a
[download] 100% of   14.32MiB in 00:00:03 at 4.30MiB/s   
[Merger] Merging formats into "/content/videos//Week 1 - Lecture 1 - Introduction to Machine Learning.mp4"
Deleting original file /content/videos//Week 1 - Lecture 1 - Introduction to Machine Learning.f60

In [ ]:
video_path = glob("/content/videos/*")

In [ ]:
video_path

['/content/videos/W1 L1 - Introduction to OS.mp4',
 '/content/videos/Week 1 - Lecture 1 - Introduction to Machine Learning.mp4',
 '/content/videos/Recoverable schdeules and transaction isolation levels.mp4',
 '/content/videos/Part1 - Definition of Information Security.mp4']

In [ ]:
output_Audio='/content/Audio/'
if not os.path.exists(output_Audio):
        os.makedirs(output_Audio)

In [ ]:

for path in video_path:
    try:
        audio_output_path = path.replace("videos", "Audio").replace(".mp4", ".mp3")

        output_dir = os.path.dirname(audio_output_path)
        os.makedirs(output_dir, exist_ok=True)

        video_clip = VideoFileClip(path)
        audio_clip = video_clip.audio

        audio_clip.write_audiofile(audio_output_path)

        audio_clip.close()
        video_clip.close()

        print(f"Audio extracted successfully for {path}.")

    except Exception as e:
        print(f"Failed to extract audio for {path}. Error: {e}")


MoviePy - Writing audio in /content/Audio/W1 L1 - Introduction to OS.mp3


MoviePy - Done.
Audio extracted successfully for /content/videos/W1 L1 - Introduction to OS.mp4.
MoviePy - Writing audio in /content/Audio/Week 1 - Lecture 1 - Introduction to Machine Learning.mp3


MoviePy - Done.
Audio extracted successfully for /content/videos/Week 1 - Lecture 1 - Introduction to Machine Learning.mp4.
MoviePy - Writing audio in /content/Audio/Recoverable schdeules and transaction isolation levels.mp3


MoviePy - Done.
Audio extracted successfully for /content/videos/Recoverable schdeules and transaction isolation levels.mp4.
MoviePy - Writing audio in /content/Audio/Part1 - Definition of Information Security.mp3


MoviePy - Done.
Audio extracted successfully for /content/videos/Part1 - Definition of Information Security.mp4.


In [ ]:
output_Audio='/content/transcript/'
if not os.path.exists(output_Audio):
        os.makedirs(output_Audio)

In [ ]:
model = whisper.load_model("base")
for i in video_path:

  result = model.transcribe(i.replace("videos","Audio").split(".")[0]+".mp3")
  df_subtitles = pd.DataFrame(result["segments"]).drop(columns=["tokens","temperature","avg_logprob","compression_ratio","no_speech_prob","id","seek"])
  df_subtitles.to_csv("/content/transcript/"+i.split("/")[-1].split(".")[0]+".csv")




100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 154MiB/s]
  checkpoint = torch.load(fp, map_location=device)

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



In [ ]:
def removeStopWords(text):
  nltk.download('stopwords')
  stop_words = set(stopwords.words("english"))
  text = text.split()
  newText = ""
  for word in text:
    if word not in stop_words:
      newText += word+" "
  return newText.strip()

def applyFuncs(Data, textLabel):
  Data[textLabel] = Data[textLabel].apply(lambda x : x.lower())
  Data[textLabel] = Data[textLabel].apply(lambda x : re.sub("[^a-z0-9\s]","",x))
  Data[textLabel] = Data[textLabel].apply(lambda x : re.sub("\s+"," ",x))
  Data[textLabel] = Data[textLabel].apply(lambda x : removeStopWords(x))
  return Data

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")



The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
  warnings.warn(



model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def summarize_text(text, chunk_size=512):
    inputs = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    summary = ""

    for chunk in inputs:
        input_ids = tokenizer.encode(chunk, return_tensors='pt')


        inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
        summary_ids = model.generate(inputs.input_ids, max_length=256, min_length=128, length_penalty=2.0, num_beams=4, early_stopping=True)

        output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        summary += output + " "

    return summary


In [ ]:
subtitles=glob("/content/transcript/*.csv")

In [ ]:
for i in subtitles:
  DATA=pd.read_csv(i)
  DATA=applyFuncs(DATA,"text")
  segment_text = ". ".join(DATA['text'].tolist())
  summary = summarize_text(segment_text)
  all_sum= summary.split(". ")
  filtered_df = DATA[DATA['text'].isin(all_sum)]
  print(filtered_df)
  all_text = DATA["text"].tolist() + all_sum
  vectorizer = TfidfVectorizer().fit_transform(all_text)
  vectors = vectorizer.toarray()
  similarity_matrix = cosine_similarity(vectors[:len(DATA)], vectors[len(DATA):])
  df=DATA.copy()
  threshold = 0.4
  df["best_match"] = ""
  df["similarity_score"] = 0.0

  for j, subtitle_vector in enumerate(similarity_matrix):
    best_match_idx = subtitle_vector.argmax()
    best_score = subtitle_vector[best_match_idx]
    if best_score > threshold:
        df.at[j, "best_match"] = all_sum[best_match_idx]
        df.at[j, "similarity_score"] = best_score

  matched_df = df[df["similarity_score"] > threshold]
  matched_df.to_csv(i.split("/")[-1].split(".")[0]+"_summary.csv")






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_

    Unnamed: 0   start     end  \
0            0    0.00   23.20   
1            1   23.20   31.32   
3            3   38.92   46.64   
4            4   46.64   53.92   
5            5   53.92   62.16   
6            6   62.16   68.68   
9            9   83.92   92.76   
10          10   92.76  102.04   
11          11  102.04  112.28   
14          14  127.72  134.92   

                                                 text  
0   hello welcome first weeks lecture course intro...  
1   systems week building platform course upon fol...  
3   os particular look operating system actually f...  
4   computer also see essential role operating sys...  
5   look computer systems think different layers r...  
6   bottom layer millions billions transistors tra...  
9   included particular layer various things like ...  
10  flops registers basic vlsi units organized var...  
11  forms build things like memory ram decode unit...  
14  purchase computer hardware could execute sever...  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /ro

Empty DataFrame
Columns: [Unnamed: 0, start, end, text]
Index: []


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /ro

Empty DataFrame
Columns: [Unnamed: 0, start, end, text]
Index: []


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /ro

     Unnamed: 0   start    end text
0             0    0.00   10.0     
1             1   10.00   12.0     
2             2   12.00   15.0     
3             3   15.00   18.0     
4             4   18.00   20.0     
..          ...     ...    ...  ...
127         127  295.28  295.5     
128         128  295.50  296.0     
129         129  296.00  296.5     
130         130  296.50  297.5     
132         132  314.50  317.5     

[131 rows x 4 columns]
